In [1]:
#%load_ext wurlitzer
#^^^ used to capture C/C++ output to be displayed in the notebook.  Does not work on Windows.

#Python 2/3 compatibility
from __future__ import print_function,division

import time
from klampt import *
from IPython.display import clear_output
from klampt import vis
from klampt.vis.ipython import Playback

world = WorldModel()
if world.loadFile("../data/tx90scenario0.xml"):
    sim = Simulator(world)
    
    #If you'd like to show the print output, comment out this line
    clear_output()
    
    vis.add("world",world)
    vis.addText("HUD1","0",position=(5,5))
    playback_widget = Playback(vis.scene())
    
    display(playback_widget)
    vis.show()
else:
    print("There was a problem loading the world file")

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

Playback(children=(HBox(children=(Button(description='Play', icon='play', style=ButtonStyle(), tooltip='Start …

KlamptWidgetAdaptor(rpc={'type': 'multiple', 'calls': [{'y': 5, 'type': 'add_text', 'x': 5, 'name': 'HUD1', 't…

In [2]:
#define your visualization and controller code here
import random

next_move_time = 2.0
move_delay = 5.0
robot = world.robot(0)    
    
def control_loop(t,controller):
    global next_move_time
    vis.addText("HUD1","%.2f"%(t,),position=(5,5))
    if t >= next_move_time:
        qmin,qmax = robot.getJointLimits()
        q = [random.uniform(a,b) for (a,b) in zip(qmin,qmax)]
        controller.setMilestone(q)
        next_move_time += move_delay
    pass


In [3]:
#this code runs the simulation and binds your code to the playback buttons
dt = 0.02


def do_reset():
    sim.reset()
    sim.updateWorld()
    global next_move_time
    next_move_time = 2.0
    vis.addText("HUD1","0",position=(5,5))

def do_advance():
    global world,sim
    if world.numRobots() > 0:
        control_loop(sim.getTime(),sim.controller(0))  #call code in the above cell
    #print("Simulating...",dt)
    sim.simulate(dt)
    sim.updateWorld()
    #print("Done.")

#this binds the playback widget buttons
playback_widget.advance = do_advance
playback_widget.reset = do_reset
playback_widget.quiet = False